## Imitation learning for CARLA Autonomous Vehicles

In [1]:
# imports
import os
import h5py
import time
import yaml
import random
import datetime
import pprint
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import torch
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms


In [2]:
# seeds
seed = 1
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)


In [3]:
# tensorboard init
date_str = datetime.datetime.now().strftime('%Y%m%d_%H%M')
writer = SummaryWriter(log_dir="./log/"+date_str)

In [4]:
# timer init
begin = time.time()

### Network Architecture

In [5]:
# network architecture
class Network(nn.Module):
    
    def __init__(self):
        super(Network, self).__init__()
        
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=5, stride=2, padding=0),
            nn.BatchNorm2d(num_features=32),
            nn.Dropout2d(),
            nn.ReLU(),)
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=1, padding=0),
            nn.BatchNorm2d(num_features=32),
            nn.Dropout2d(),
            nn.ReLU(),)
        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=2, padding=0),
            nn.BatchNorm2d(num_features=64),
            nn.Dropout2d(),
            nn.ReLU(),)
        self.conv4 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=0),
            nn.BatchNorm2d(num_features=64),
            nn.Dropout2d(),
            nn.ReLU(),)
        self.conv5 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=2, padding=0),
            nn.BatchNorm2d(num_features=128),
            nn.Dropout2d(),
            nn.ReLU(),)
        self.conv6 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=0),
            nn.BatchNorm2d(num_features=128),
            nn.Dropout2d(),
            nn.ReLU(),)
        self.conv7 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=0),
            nn.BatchNorm2d(num_features=256),
            nn.Dropout2d(),
            nn.ReLU(),)
        self.conv8 = nn.Sequential(
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=0),
            nn.BatchNorm2d(num_features=256),
            nn.Dropout2d(),
            nn.ReLU(),)
        self.flatten = nn.Flatten()
        self.dense1 = nn.Sequential(
            nn.Linear(in_features=8192, out_features=512),
            nn.ReLU())
        self.dense2 = nn.Sequential(
            nn.Linear(in_features=512, out_features=512),
            nn.ReLU())
        self.output = nn.Linear(512, 2)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.conv6(x)
        x = self.conv7(x)
        x = self.conv8(x)
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dense2(x) 
        x = self.output(x)
        return x

In [6]:
# model init
model = Network().cuda()

In [7]:
# optimizer and criterion init
optimizer = torch.optim.Adam(model.parameters(),lr=0.0001)
criterion = nn.MSELoss()

### Dataset Preparation

In [8]:
train_path = os.path.join(os.getcwd(), 'CORL2017ImitationLearningData/AgentHuman/SeqTrain')
test_path  = os.path.join(os.getcwd(), 'CORL2017ImitationLearningData/AgentHuman/SeqVal')
train_dataset_ls = os.listdir(path=train_path) # 3288
num_train_dataset = len(train_dataset_ls)
test_dataset_ls = os.listdir(path=test_path)   # 374
num_test_dataset = len(test_dataset_ls)

print('# train h5:', num_train_dataset)
print('# test  h5:', num_test_dataset)

# train h5: 3288
# test  h5: 374


### Parameter setup

In [9]:
num_epoch = 200
batch_size = 200

### Training and validation loop

In [10]:
for epoch in range(num_epoch):
    # print("Epoch:%4d"  % epoch)
    train_loss = 0
    test_loss = 0
    best_loss = 1
    
    # training 
    for file_idx in range(num_train_dataset): #num_file):
        # a file as a batch
        filename = os.path.join(train_path, train_dataset_ls[file_idx])
        f = h5py.File(filename, 'r')
        # print(list(f.keys()))
        img = torch.Tensor(np.array(f['rgb'])).permute(0,3,2,1).cuda()
        tar = torch.Tensor(np.array(f['targets'])).cuda()
        
        steer, throttle, brake = tar[:,0].unsqueeze(1), tar[:,1].unsqueeze(1), tar[:,2].unsqueeze(1)
        ctl = torch.hstack((steer, throttle-brake))

        f.close()
        
        model.train(True)
        optimizer.zero_grad()       # zero the parameter gradients
        out = model(img)
        loss = criterion(out, ctl)  # compute the loss 
        loss.backward()             # backpropagate the loss
        optimizer.step()            # adjust parameters based on the calculated gradients
        train_loss += loss.item() 
        # if file_idx % 100 == 0:
            # print(f'Epoch: {epoch:2d} [{file_idx:4d} {num_train_dataset:4d}] loss={loss.item():.5f}')
        
    writer.add_scalar('MSE_loss/train', train_loss/num_train_dataset, epoch)
    if best_loss > train_loss/num_train_dataset:
        torch.save(model.state_dict(), "./model/model_weights_"+date_str+'.pth')
        torch.save(model, "./model/model_"+date_str+'.pth')
        
    # testing
    for file_idx in range(num_test_dataset): #num_file):
        # a file as a batch
        filename = os.path.join(train_path, train_dataset_ls[file_idx])
        # print(filename)
        f = h5py.File(filename, 'r')
        # print(list(f.keys()))
        img = torch.Tensor(np.array(f['rgb'])).permute(0,3,2,1).cuda()
        tar = torch.Tensor(np.array(f['targets'])).cuda()
        
        f.close()
        
        steer, throttle, brake = tar[:,0].unsqueeze(1), tar[:,1].unsqueeze(1), tar[:,2].unsqueeze(1)
        ctl = torch.hstack((steer, throttle-brake))
        model.train(False)
        model.eval()
        out = model(img)
        loss = criterion(out, ctl)
        test_loss += loss.item()
        # if file_idx % 200 == 0:
            # print(f'Epoch: {epoch:2d} [{file_idx:4d} {num_test_dataset:4d}] loss={loss.item():.5f}')
        
    writer.add_scalar('MSE_loss/test', test_loss/num_test_dataset, epoch)
    writer.add_scalars('MSE_loss', {'train_loss':  train_loss/num_train_dataset, 
                                    'test_loss' :  test_loss /num_test_dataset   }
                      ,epoch)
    
    print(f"Epoch: {epoch:2d} Loss/train: {train_loss/num_train_dataset} Loss/test: {test_loss/num_test_dataset}")

Epoch:  0 Loss/train: 0.16378255382873042 Loss/test: 0.1697624755131808
Epoch:  1 Loss/train: 0.1400976739197838 Loss/test: 0.15276352897595197
Epoch:  2 Loss/train: 0.12996797470769328 Loss/test: 0.15065172588935788
Epoch:  3 Loss/train: 0.1240373366757227 Loss/test: 0.1494168984382488
Epoch:  4 Loss/train: 0.11913408777521302 Loss/test: 0.1473292045066143
Epoch:  5 Loss/train: 0.11488240469540832 Loss/test: 0.1364918893419784
Epoch:  6 Loss/train: 0.11076546749523546 Loss/test: 0.13302829428476326
Epoch:  7 Loss/train: 0.10710594050976824 Loss/test: 0.13014971467580416
Epoch:  8 Loss/train: 0.10415182203013923 Loss/test: 0.1226813797659616
Epoch:  9 Loss/train: 0.10136085946741401 Loss/test: 0.11726346389674719
Epoch: 10 Loss/train: 0.09858118187897576 Loss/test: 0.1130536557052267
Epoch: 11 Loss/train: 0.0960742118970917 Loss/test: 0.11142298240817829
Epoch: 12 Loss/train: 0.09370006654803005 Loss/test: 0.10804774802623884
Epoch: 13 Loss/train: 0.09147953220423302 Loss/test: 0.10538

In [11]:
end = time.time()
print("Goodbye Sir!", end - begin)

Goodbye Sir! 23510.922435760498
